In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

import re
import requests
import datetime
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from konlpy.tag import Komoran

from sklearn.manifold import TSNE
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

# Crawling 

In [12]:
def clean_html(x):
  x = re.sub("\&\w*\;","",x)
  x = re.sub("<.*?>","",x)
  return x


search_category = ['정치', '경제', '사회', '생활/문화', '세계', 'IT/과학']

client_id = "cByyPKiwDvI4YNRsAA4g" #1.에서 취득한 아이디 넣기
client_secret = "9v0xRpUYvj"  #1. 에서 취득한 키 넣기

search_word = 'IT' #검색어
encode_type = 'json' #출력 방식 json 또는 xml
max_display = 100 #출력 뉴스 수
sort = 'date' #결과값의 정렬기준 시간순 date, 관련도 순 sim
start = 1 # 출력 위치

url = f"https://openapi.naver.com/v1/search/news.{encode_type}?query={search_word}&display={str(int(max_display))}&start={str(int(start))}&sort={sort}"

#헤더에 아이디와 키 정보 넣기
headers = {'X-Naver-Client-Id' : client_id,
           'X-Naver-Client-Secret':client_secret
           }

r = requests.get(url, headers=headers)
#요청 결과 보기 200 이면 정상적으로 요청 완료
print(r)

r.json()

df = pd.DataFrame(r.json()['items'])
df['title'] = df['title'].apply(lambda x: clean_html(x))
df['description'] = df['description'].apply(lambda x: clean_html(x))

df.to_csv(f'news_search_result_{search_word}.csv', encoding = "utf-8-sig")

<Response [200]>


In [53]:
sid = ['100', '101', '102', '103', '104', '105']

def get_soup_obj(url):
    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36" }
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup

default_img = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#"
df = pd.DataFrame()

for s in sid : 
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=" + s
    
    print("section url : ", sec_url)
    
    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)
  
    # 해당 분야 상위 뉴스 10개 가져오기
    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=100)
    for li in lis3:
        # title : 뉴스 제목, news_url : 뉴스 URL, image_url : 이미지 URL
        news_info = {
            "title" : li.img.attrs.get('alt') if li.img else li.a.text.replace("\n", "").replace("\t","").replace("\r","") , 
            "date" : li.find(class_="date").text,
            "news_url" : li.a.attrs.get('href'),
            "image_url" :  li.img.attrs.get('src') if li.img else default_img,
            "category" : s
        }
        
        print(news_info)
        df = df.append(news_info, ignore_index=True)
    
df.to_csv('test.csv', encoding = "utf-8-sig")

section url :  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=100
{'title': '[연합시론] 청년층 경제적고통 최악이라는데, 대선주자들 분발해야', 'date': '1분전', 'news_url': 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=001&aid=0012791479', 'image_url': 'https://imgnews.pstatic.net/image/origin/001/2021/11/14/12791479.jpg?type=ofullfill106_72', 'category': '100'}
{'title': '野 "이재명, 부산이 우습냐" vs 與 "부산 발전 위해 뭐했나"', 'date': '2분전', 'news_url': 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=003&aid=0010831137', 'image_url': 'https://imgnews.pstatic.net/image/origin/003/2021/11/14/10831137.jpg?type=ofullfill106_72', 'category': '100'}
{'title': '한-카메룬 고위급 정책협의회…ILO출마 강경화 지지요청', 'date': '2분전', 'news_url': 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=001&aid=0012791478', 'image_url': 'https://imgnews.pstatic.net/image/origin/001/2021/11/14/12791478.jpg?type=ofullfill106_72', 'category': '100'}
{'title': '한국시리즈 관람하는 윤석열', 'date': '2분전', 'news_u

{'title': '"가을 끝자락 낭만 만끽"…전국 명산·관광지 행락객 운집', 'date': '3분전', 'news_url': 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=103&oid=001&aid=0012791477', 'image_url': 'https://imgnews.pstatic.net/image/origin/001/2021/11/14/12791477.jpg?type=nf106_72', 'category': '103'}
{'title': "제주항공, 12월부터 '부산~사이판' 운항 재개…매주 수요일 '1회'", 'date': '3분전', 'news_url': 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=103&oid=079&aid=0003574618', 'image_url': 'https://imgnews.pstatic.net/image/origin/079/2021/11/14/3574618.jpg?type=nf106_72', 'category': '103'}
{'title': "현대차그룹, '2021 뉴스위크 오토 어워지' 7개 부문 수상", 'date': '5분전', 'news_url': 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=103&oid=079&aid=0003574616', 'image_url': 'https://imgnews.pstatic.net/image/origin/079/2021/11/14/3574616.jpg?type=nf106_72', 'category': '103'}
{'title': "[가족] 올무줄 걸린 채 떠돌던 유기견 '장군이'의 변신", 'date': '6분전', 'news_url': 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=103&oid=469&aid

PermissionError: [Errno 13] Permission denied: 'test.csv'